数据封装、继承和多态是面向对象的基本特性，但是还有很多高级特性需要学习

## 使用__slots__

我们定义好class，创建对应的实例之后，可以给该实例绑定任何的属性和方法，这是动态语言的灵活性。

In [3]:
class Student(object):
    pass

In [6]:
def set_age(self, age):
    self.age = age

from types import MethodType 

s = Student()
s.set_age = MethodType(set_age, s)
s.set_age(25)
print(s.age)

25


In [7]:
s2 = Student()
s2.set_age(10)

AttributeError: 'Student' object has no attribute 'set_age'

In [9]:
def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

In [11]:
s.set_score(100)
s.score

100

In [14]:
s2.set_score(99)
s2.score

99

In [23]:
class Student(object):
    __slots__ = ('name', 'age')

In [24]:
s = Student()
s.age = 25
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

In [25]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 9999

## 使用@property

绑定属性的时候，如果我们直接把属性暴露出去，虽然写起来简单，但是没办法修改检查参数，导致属性可以随便改。
这个时候可以用@property把方法变成属性
把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身会创造另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，具体操作如下

In [1]:
class Student(object):

    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must be between 0~100')
        self._score = value

In [2]:
s = Student()
s.score = 60
s.score = 999

ValueError: score must be between 0~100

可以只定义getter方法，不定义setter方法，这样就是一个只读属性
另外需要注意属性的方法名和实例变量名不要重名

In [4]:
class Student(object):

    # 方法名称和实例变量均为birth:
    @property
    def birth(self):
        return self.birth ## 错误示范

#### 小结
@property 广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，这样程序运行可以减少出错的可能性。